In [1]:
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

# 1. Análisis de personas distintas atendidas por diagnóstico

En este cuadernillo se quieren responder las siguiente preguntas:

1. ¿Cuántas personas fueron atendidas para cada uno de los diagnósticos del INT?


In [2]:
# Lee solamente la base del INT
ANIOS_A_ANALIZAR = [2015, 2016, 2017, 2018, 2019]

df_torax = (
    pl.scan_csv("../data/processed/egresos_procesados.csv")
    .filter(
        (pl.col("ESTABLECIMIENTO_SALUD") == 112103) & (pl.col("ANO_EGRESO").is_in(ANIOS_A_ANALIZAR))
    )
    .collect(streaming=True)
)

Los problemas al momento de calcular la cantidad de pacientes distintos para un problema de salud son los siguientes:

1. Un paciente puede tener egresos hospitalarios con distintos diagnosticos que apuntan a un mismo problema de salud (Ej: Diagnostico E840, E841, E849 apuntar al problema de salud "Fibrosis Quistica"). Esto es un problema, ya que el paciente sera considerado como un paciente distinto dependiendo del diagnostico asignado. A modo de ilustracion, se tiene el siguiente ejemplo:

   - Paciente X diagnosticado con E840 y E841 en un mismo año. Esto significaría que hay 1 paciente distinto para E840 y E841. Sin embargo, si analizamos el problema de salud "Fibrosis Quística", solamente existiría 1 paciente distinto (y no 2).

Debido a lo anterior, es necesario tomar las precauciones necesarios al hacer los calculos.


In [3]:
# Define diagnosticos para los problemas de salud de interes
PROBLEMAS_DE_SALUD = {
    "FIBROSIS": ["E840", "E841", "E849", "E848"],
    "HIPERTENSION_PULMONAR": ["I270", "I272"],
    "FIBROSIS_PULMONAR": ["J841"],
    "ESCLERODERMIA": ["M348"],
    "MARFAN": ["Q874"],
    "EXCAVATUM": ["Q676"],
    "CARINATUM": ["Q677"],
    "SILICOSIS": ["J678", "J679"],
    "CARDIOPATIA_CONGENITA_ADULTO": [],
    "TRASPLANTE_DE_PULMON": [],
    "ECMO": [],
    "CANCER_DE_PULMON": [
        "C33X",
        "C340",
        "C341",
        "C342",
        "C343",
        "C381",
        "C384",
        "C450",
        "C780",
        "C782",
        "D381",
    ],
    "CIRUGIA_CANCER_DE_PULMON": [],
    "TRASPLNATE_DE_CORAZON": [],
    "ENFERMEDAD_CORONARIA": [],
    "ANEURISMA_DE_LA_AORTA": ["I712"],
    "DISECCION_DE_LA_AORTA": ["I710"],
    "MARCAPASOS": ["Z450"],
    "INSUFICIENCIA_MITRAL": ["I340"],
    "INSUFICIENCIA_AORTICA": ["I351"],
    "COVID": ["U071"],
    "APNEA_DEL_SUENO": ["G473"],
}

In [6]:
resultados = {}
for problema_de_salud, codigos_diagnostico in PROBLEMAS_DE_SALUD.items():
    # Filtra el df del torax solamente a los diagnosticos del problema
    df_diagnostico = df_torax.filter(pl.col("DIAG1").is_in(codigos_diagnostico))

    # Calcula metricas totales para el periodo analizado
    egresos_totales = df_diagnostico.shape[0]
    pacientes_totales = df_diagnostico.select(pl.col("ID_PACIENTE").n_unique()).item(0, 0)

    # Calcula metricas desagregadas por anios
    resumen_diagnostico_agrupado = (
        df_diagnostico.group_by(pl.col(["ANO_EGRESO"]))
        .agg(n_egresos=pl.col("DIAG1").count(), n_pacientes=pl.col("ID_PACIENTE").n_unique())
        .sort(pl.col("ANO_EGRESO"))
        .mean()
    )
    # Selecciona la primera fila y la segunda columna del resumen ("n_egresos")
    egresos_promedios = resumen_diagnostico_agrupado.item(0, 1)
    pacientes_promedio = resumen_diagnostico_agrupado.item(0, 2)

    # Genera el string resumen de metricas totales y promedio
    linea_resumen_egresos = f"{egresos_totales} ({egresos_promedios})"
    linea_resumen_pacientes = f"{pacientes_totales} ({pacientes_promedio})"

    # Guarda los strings resumen en un DataFrame
    resumen_problema_salud = pd.DataFrame(
        {
            "egresos_totales_2015_a_2019 (suma - promedio)": [linea_resumen_egresos],
            "personas_atendidas_2015_a_2019 (suma - promedio)": [linea_resumen_pacientes],
        },
        index=[problema_de_salud],
    )

    resultados[problema_de_salud] = resumen_problema_salud

In [7]:
resumen_total_diags = pd.concat(resultados.values())
display(resumen_total_diags)

,egresos_totales_2015_a_2019 (suma - promedio),personas_atendidas_2015_a_2019 (suma - promedio)
FIBROSIS,224 (44.8),65 (25.2)
HIPERTENSION_PULMONAR,111 (22.2),90 (18.8)
FIBROSIS_PULMONAR,601 (120.2),460 (101.8)
ESCLERODERMIA,104 (20.8),73 (16.2)
MARFAN,2 (1.0),2 (1.0)
EXCAVATUM,53 (10.6),39 (8.6)
CARINATUM,20 (4.0),18 (3.6)
SILICOSIS,323 (64.6),230 (51.8)
CARDIOPATIA_CONGENITA_ADULTO,0 (None),0 (None)
TRASPLANTE_DE_PULMON,0 (None),0 (None)


In [9]:
resumen_total_diags.to_excel("../data/interim/resumen_area_influencia_hospitalizado.xlsx")